# GPT-Neo Models

<a href="https://colab.research.google.com/github/hjesse92/style_transfer_w266/blob/main/notebooks/GPT_Neo_Models.ipynb" target="_parent"><img src="https://
colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [28]:
!pip install -q transformers rouge_score evaluate

  Preparing metadata (setup.py) ... done


In [2]:
#Am I running a GPU and what type is it?
!nvidia-smi

Wed Mar 22 16:38:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch

# Clear out cuda
torch.cuda.empty_cache()

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('Number of GPU(s) available:', torch.cuda.device_count())
    print('GPU device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available')
    device = torch.device("cpu")

Number of GPU(s) available: 1
GPU device name: Tesla T4


In [4]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import GPTNeoForCausalLM, AutoTokenizer
# from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamW, Seq2SeqTrainingArguments, Seq2SeqTrainer

import re
import random
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pprint
import nltk

from logging import warning
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd drive/MyDrive/w266/style_transfer_w266/

/content/drive/MyDrive/w266/style_transfer_w266


In [7]:
train_file = 'data/original-train.tsv'
dev_file = 'data/original-dev.tsv'
test_file = 'data/original-test.tsv'
df_train = pd.read_csv(train_file, sep='\t')
df_dev = pd.read_csv(dev_file, sep='\t')
df_test = pd.read_csv(test_file, sep='\t')

## Few Shot Learning GPT-Neo 1.3B

In [8]:
#@title 3-Shot Learning
df_fewshot = df_test.copy()
n_shots = 3

for row in range(len(df_fewshot)):
    #For each item in the test set, we use 3 examples from the training set to as n-shots
    sources_targets = df_train.sample(n_shots, replace=False)

    for i in range(n_shots):
        df_fewshot.loc[row, f'shot{i+1}_source'] = sources_targets.iloc[i,0]
        df_fewshot.loc[row, f'shot{i+1}_target'] = sources_targets.iloc[i,1]


df_fewshot = df_fewshot.rename(columns={'offensive-text':'source', 'style-transferred-text':'target'})
df_fewshot.head()

,source,target,shot1_source,shot1_target,shot2_source,shot2_target,shot3_source,shot3_target
0,So maybe you should be more retarded.,So maybe you should be more backward,Has been a US military puppet from jump,Has been a tool for the US military from the b...,It's not though you fucking idiot,It's not though,Maybe you shouldn't turn your new country into...,You should be careful not to bring your old pr...
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,wasn't that like 12 feds and an autistic fuck,wasn't that like 12 feds and an unintelligent ...,"Sympathy: That sucks, I'm sorry you feel that ...","Sympathy: That stinks, I'm sorry you feel that...","Yep, it sucks...","Yeah, that's bad"
2,the fuck.... you are on drugs,uh..... you are not being realistic,This guy is overrated..... But he is right abo...,This guy is overrated...... But he is right ab...,That roommate's presence is gross and we need ...,That roommate's presence is unbearable and we ...,Funny how Biden canceled Trump’s insulin cap w...,Funny how Biden canceled Trump’s insulin cap w...
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,You don’t deserve it dumbass.,You don't deserve it.,I just realized you are from India LMAO bet yo...,I just realized you are from India LMAO are yo...,Also sounds like facism,Also sounds like autocracy
4,Youre soft as baby shit,Youre really soft,"Don't dare say a word, whitey. We run this lib...",Don't dare say a word. This sub is liberal.,Yep my Karma is going way down 😆 and my masoch...,Yep my Karma is going way down 😆 and my pain l...,"Pre-Covid Libtards: My body, my choice! Black...","Pre-Covid Liberals: My body, my choice! Black ..."


In [9]:
# df_fewshot['prompt'] = df_fewshot.apply(lambda x: 
#                  'Rewrite the toxic text in non-toxic style: \n\n'
#                  'Toxic text: ' + x['shot1_source'] + '\n' + 'Non-toxic text: ' + x['shot1_target'] + '\n\n' + \
#                  'Toxic text: ' + x['shot2_source'] + '\n' + 'Non-toxic text: ' + x['shot2_target'] + '\n\n' + \
#                  'Toxic text: ' + x['shot3_source'] + '\n' + 'Non-toxic text: ' + x['shot3_target'] + '\n\n' + \
#                  'Toxic text: ' + x['source'] + '\n' + 'Non-toxic text: ', axis=1)

df_fewshot['prompt'] = df_fewshot.apply(lambda x: 
                 'Rewrite the toxic text in non-toxic style: \n###\n'
                 'Toxic text: ' + x['shot1_source'] + '\n' + 'Non-toxic text: ' + x['shot1_target'] + '\n' + '###' + '\n' +\
                 'Toxic text: ' + x['shot2_source'] + '\n' + 'Non-toxic text: ' + x['shot2_target'] + '\n' + '###' + '\n' +\
                 'Toxic text: ' + x['shot3_source'] + '\n' + 'Non-toxic text: ' + x['shot3_target'] + '\n' + '###' + '\n' +\
                 'Toxic text: ' + x['source'] + '\n' + 'Non-toxic text: ', axis=1)

In [10]:
print(df_fewshot['prompt'][0])

Rewrite the toxic text in non-toxic style: 
###
Toxic text: Has been a US military puppet from jump
Non-toxic text: Has been a tool for the US military from the beginning
###
Toxic text: It's not though you fucking idiot
Non-toxic text: It's not though
###
Toxic text: Maybe you shouldn't turn your new country into the shithole you just left.
Non-toxic text: You should be careful not to bring your old problems from your old country to your new country.
###
Toxic text: So maybe you should be more retarded.
Non-toxic text: 


In [11]:
neomodel = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
neomodel.to(device)
neotokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
neotokenizer.pad_token = neotokenizer.eos_token

In [13]:
for i in range(len(df_fewshot)):
    if i%10 == 0:
        print(f'Working on number: {i}')
    neo_input = neotokenizer([df_fewshot.prompt[i]], return_tensors='pt', 
                             )
    neo_input_ids = neo_input.input_ids.to(device)
    neo_input_mask = neo_input.attention_mask.to(device)

    generated_ids = neomodel.generate(input_ids=neo_input_ids, 
                                      attention_mask=neo_input_mask,
                                      num_beams=5,
                                      no_repeat_ngram_size=2,
                                      num_return_sequences=1,
                                      top_p = 0.92,
                                      top_k = 50,
                                      max_length = 512,
                                      do_sample=True,
                                      temperature=0.9,
                                      early_stopping=True,
                                      )
    output = neotokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    df_fewshot.loc[i,'fewshot_output'] = output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Working on number: 0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Working on number: 10


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Working on number: 180


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Working on number: 190


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [14]:
df_fewshot.loc[:,['source','target','prompt','fewshot_output']]

,source,target,prompt,fewshot_output
0,So maybe you should be more retarded.,So maybe you should be more backward,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
2,the fuck.... you are on drugs,uh..... you are not being realistic,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
4,Youre soft as baby shit,Youre really soft,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
...,...,...,...,...
194,"NTA, Jes-us your brother is dumb. How long did...","NTA, your brother is not thinking straight. Ho...",Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
195,Formula one…nascar….shit it exactly the same e...,Formula one…nascar….it exactly the same except...,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
196,"BRB, gonna go call the mods pathetic egotistic...","BRB, gonna go call the mods.",Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...
197,CUCKOLD Carlson is a problem,Carlson is the problem,Rewrite the toxic text in non-toxic style: \n#...,Rewrite the toxic text in non-toxic style: \n#...


In [15]:
print(df_fewshot['fewshot_output'][0])

Rewrite the toxic text in non-toxic style: 
###
Toxic text: Has been a US military puppet from jump
Non-toxic text: Has been a tool for the US military from the beginning
###
Toxic text: It's not though you fucking idiot
Non-toxic text: It's not though
###
Toxic text: Maybe you shouldn't turn your new country into the shithole you just left.
Non-toxic text: You should be careful not to bring your old problems from your old country to your new country.
###
Toxic text: So maybe you should be more retarded.
Non-toxic text:  You need to learn how to use your brain before you can use it to make your country a better place.


In [16]:
import re

def extract_predicted_text(x):
  # Remove the first 3 shot examples & instruction
  few_shot_pred = x.split('\n###\n')[4]
  # Extract output text from GPT-Neo
  few_shot_pred = re.findall("\nNon-toxic text: (.*)", few_shot_pred)[0]
  return few_shot_pred

In [17]:
df_fewshot['fewshot_output_text'] = df_fewshot['fewshot_output'].apply(extract_predicted_text)

In [18]:
df_fewshot.loc[:,['source','target','prompt','fewshot_output_text']]

,source,target,prompt,fewshot_output_text
0,So maybe you should be more retarded.,So maybe you should be more backward,Rewrite the toxic text in non-toxic style: \n#...,You need to learn how to use your brain befor...
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,Rewrite the toxic text in non-toxic style: \n#...,I don't want to talk about that shit here. I ...
2,the fuck.... you are on drugs,uh..... you are not being realistic,Rewrite the toxic text in non-toxic style: \n#...,the fucks.... are you on any drugs?
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,Rewrite the toxic text in non-toxic style: \n#...,N-T-A-L-E-R-S-I-N-G-O-F-C-U-M-B-Y-P-H
4,Youre soft as baby shit,Youre really soft,Rewrite the toxic text in non-toxic style: \n#...,You're a fucking piece of shit and I hate you
...,...,...,...,...
194,"NTA, Jes-us your brother is dumb. How long did...","NTA, your brother is not thinking straight. Ho...",Rewrite the toxic text in non-toxic style: \n#...,"NPA, you are too stupid to know what you're t..."
195,Formula one…nascar….shit it exactly the same e...,Formula one…nascar….it exactly the same except...,Rewrite the toxic text in non-toxic style: \n#...,"Formula 1, nascar, shit it’s the exact same b..."
196,"BRB, gonna go call the mods pathetic egotistic...","BRB, gonna go call the mods.",Rewrite the toxic text in non-toxic style: \n#...,"BRb, gotta go to the bathroom. Gotta take a s..."
197,CUCKOLD Carlson is a problem,Carlson is the problem,Rewrite the toxic text in non-toxic style: \n#...,Cuckold Carlson IS A PROBLEM


In [25]:
for i in range(10):
  print(i, ": Original Text:", df_fewshot['source'][i])
  print (i, ": Output Text:",df_fewshot['fewshot_output_text'][i])

0 : Original Text: So maybe you should be more retarded.
0 : Output Text:  You need to learn how to use your brain before you can use it to make your country a better place.
1 : Original Text: THERES A MEGATHREAD FOR VACCINE OR COVID RELATED TOPICS. DON'T TALK ABOUT THAT SHIT HERE IDIOT!
1 : Output Text:  I don't want to talk about that shit here. I want you to listen to what I have to say, not what you think I should or shouldn't be saying. You're not the boss of me, you're just the messenger. That's all.
2 : Original Text: the fuck.... you are on drugs
2 : Output Text:  the fucks.... are you on any drugs?
3 : Original Text: NTA.   Dump his ass. Tablemanners are no rocket science. Treating other people like human beings is no rocket science. His still a child.
3 : Output Text:  N-T-A-L-E-R-S-I-N-G-O-F-C-U-M-B-Y-P-H
4 : Original Text: Youre soft as baby shit
4 : Output Text:  You're a fucking piece of shit and I hate you
5 : Original Text: Because everyone knows why. It’s jadas fault. 

In [19]:
df_fewshot = df_fewshot.loc[:,['source','target','prompt','fewshot_output_text']]

In [26]:
df_fewshot.to_csv('outputs/neo_few_shot_output2.csv',sep='\t',index=False)

### Evaluation with Rouge

In [29]:
import evaluate

rouge = evaluate.load('rouge')

In [30]:
#@title Score after few shot learning
print(rouge.compute(predictions=df_fewshot.fewshot_output_text,
              references=df_fewshot.target))

{'rouge1': 0.23098823691473924, 'rouge2': 0.06170245504955012, 'rougeL': 0.20364669294185744, 'rougeLsum': 0.20412343983854597}


### Evaluation with NonToxicScore

In [31]:
import sys
sys.path.append('./notebooks')
from DistilBertClassification import BertClassificationML, NonToxicScoreDataLoader, NonToxicScore

# Load DistilBERT Classification Model to calculate NonToxicScore
score_model = BertClassificationML()
score_model = score_model.to(device)

# Load training weights
pretrained_weights = torch.load('models/DistilBertToxicClassification5.pth')
score_model.load_state_dict(pretrained_weights )

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [32]:
output_file = 'outputs/neo_few_shot_output2.csv'
output_col = 'fewshot_output_text'

# Create Data Loader
score_loader = NonToxicScoreDataLoader(output_file, output_col)

# Calculate NonToxicScore
fewshot_NonToxicScores, avg_score = NonToxicScore(score_loader, score_model)

{'NonToxicScore': 0.3271968613317863}


## Fine Tuning GPT-Neo